In [1]:
import pandas as pd
import numpy as np
import cv2
import gc

In [2]:
SIZE = 128

def just_resize(img0, size=SIZE):
    return cv2.resize(img0,(size,size))

In [3]:
train_df = pd.read_csv('train.csv')
n_image = len(train_df)
n_file = 4

In [4]:
k = 0
train_idx = []
train_values = np.zeros((n_image, 137 * 236), dtype = "uint8")
for i in range(n_file):
    print("reading train file", i)
    directory = "DATA/train_image_data_"+str(i)+".parquet"
    train_f = pd.read_parquet(directory, engine = "pyarrow")
    train_f.set_index('image_id', inplace=True)
    train_idx.append(train_f.index.copy())
    n_file_image = len(train_f)
    train_values[i * n_file_image : (i + 1) * n_file_image, :] = 255 - train_f.values
    del train_f
    gc.collect()

train_values = train_values.reshape((-1, 137, 236))
np.save('train_2.npy', train_values)

reading train file 0
reading train file 1
reading train file 2
reading train file 3


## RESIZE

In [ ]:
train_values = np.load('train_2.npy')

In [12]:
train_values_resized = []
for i, img in enumerate(train_values):
    #img = (img*(255.0/img.max())).astype(np.uint8)
    #img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    resized_img = just_resize(img)
    train_values_resized.append(resized_img)

In [13]:
train_values_resized = np.array(train_values_resized)

In [14]:
np.save('train_resized.npy', train_values_resized)

## Read Images

In [ ]:
def new_open_image(fn:PathOrStr, div:bool=True, convert_mode:str='L',
                   after_open:Callable=None, transforms=True)->Image:
    file_index = int(fn.split('/')[-1])
    x = train_images[file_index]
    if after_open: x = after_open(x)
        
    if transforms:
        transformed_im = train_tf(image=x)['image']
        x = pil2tensor(transformed_im, dtype=np.float32)
    else:
        x = pil2tensor(x, dtype=np.float32)
    if div: x.div_(255)
    return vision.Image(x)